In [ ]:
# run this cell
from datascience import *
import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Lab 4: Joining Data

1. [Preparing Data](#prep)
2. [Joining Labs 1 and 3](#lab)
3. [Joining Forgotten Data](#forgotten)
4. [Joining Other Data](#outside)

In last week's lab, we prepared data on children's height-for-age zscores & other anthropometric measures to join to data from the end of Lab 1.  Here we will prepare our Lab 1 data for joining, just as we did in Lab 3. Read in your data from Lab 1.

In [ ]:
roster = ...
roster
### STUDENT SOLUTION
roster = Table.read_table('2-06/Data_L1.csv')

## Preparing Data <a id='prep'></a>

We want to join the tables, linking with household id and individual id.  However, these are currently two separate columns.  We need to combine them into a single Master Id.

We need to assign a unique value to each pair of the form `(<household id>, <individual id>)`. One way to do this is to construct a number of the form `<household id>0<individual_id>`. Here, the 0 acts as a separator, telling us where the household id ends and where the individual id begins. A method to get a number of this form is to use the following equation: 

$$id = household\_id * 10^n + individual\_id$$ 

where $n$ is larger than the number of digits in the largest individual id. (Exercise: think about why this is true). 

We have provided the rough code to create a table with the Master ID column. However, you need to fill out the value of n. Use the blank cell below to find out how many digits there are in the largest individual id in your roster.

In [ ]:
### STUDENT SOLUTION
max(roster["Individual ID"])

Two things before you proceed with the next cell:

1. Your Individual ID column may already a unique identifier. To check if that is the case, group on Individual ID and check if all values in the `count` column equal to one. If that is the case, you do not have to create a Master ID column, and can join with just the Individual ID column.

2. `roster['Household ID']` is almost equivalent to `roster.column('Household ID')`. Both will return an array of the values in a column, but when we use `roster['Household ID']`, we are also able to directly access the values in that table. `roster.column('Household ID')` makes a copy of the data, so we can't accidentally change original values of the table.

Because we are able to directly access the values in the table, we can make inplace assignments. Compare the two lines of code below, which effectively do the same thing.

    roster = roster.with_column('Master ID', roster.column('Household ID')*n + roster.column('Individual ID'))
    roster['Master ID'] = roster['Household ID']*n + roster['Individual ID']
    
With `datascience`, this shorthand method is generally considered bad practice, but from time to time, you'll see it written this way (mainly by me).

In [ ]:
n = ...
roster['Master ID'] = roster['Household ID']*n + roster['Individual ID']
roster
### STUDENT SOLUTION
n = 100

## Joining Tables <a id='lab'></a>

Read in your data from Lab 3. We are going to join that information with our roster data.

In [ ]:
zscores = ...
zscores
### STUDENT SOLUTION
zscores = Table.read_table("2-20/Lab_3.csv")
zscores

The `join` method in `datascience` performs an inner join. That means that it joins two tables based on the set of records that match in both tables. So for example, if we join our `zscores` and `roster` on `Master ID`, we will get rows for each instance where a `Master ID` occurs in both tables. That means that if a `Master ID` is not in both tables, all information about that ID is lost.

![](joins.jpg)

We will want to perform a left join. A left join maintains a complete set of rows from the left table, combined with matching records in the right table. If the left value doesn't have a corresponding row in the right table, the unknown values from the join will take a null value. The diagram above is a pictorial representation of different types of joins.

In the cell below, perform a `datascience` join between your Lab 1 and Lab 3 data sets, and check how many rows you have.

In [ ]:
### STUDENT SOLUTION
roster.join('Master ID', zscores)

Unless your roster consists solely of children under the age of 5, that table should have less rows than your Lab 1 data and the same number of rows as your Lab 3 data.

We are going to need to use `pandas` again. You used it in Lab 1 to fill in missing values (which you will do again later in this lab), but will be using it right now for left joins. You are able to convert a table to the `pandas` version of a table (called a dataframe) with the `to_df` method.

For right now, we are going to use the `pandas` `merge` function to join our tables together. Here is how we are going to do this:

1. Convert our tables to dataframes using `.to_df` ([docs](http://data8.org/datascience/_autosummary/datascience.tables.Table.to_df.html?highlight=to_df#datascience.tables.Table.to_df))
2. Use the `merge` function ([docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)) with the parameters `how='left'` (which specifies the type of join) and `on=['Master ID']` (which specifies a list of columns to join on).  Unlike in the datascience package, these variables must have the same name.
3. Create a datascience table from that new dataframe using `.from_df` ([docs](http://data8.org/datascience/_autosummary/datascience.tables.Table.from_df.html?highlight=from_df#datascience.tables.Table.from_df))

### <font color="blue"> Item 1: Paste the line(s) of code that joins the data into your google doc.  Explain why you chose the order of the data sets in that line of code.

In [ ]:
combined = ...
### STUDENT SOLUTION
combined = Table.from_df(pd.merge(roster.to_df(), zscores.to_df(), how='left', on=['Master ID']))
combined

### <font color="blue"> Item 2: Sanity Check
 <font color="blue"> a) Are there still the same number of people in your data as on the original roster (Lab 1)? You can use the cell below to check with code.

In [ ]:
### STUDENT SOLUTION
combined.num_rows == roster.num_rows

<font color="blue"> b) If you examine the new zscore variables (Lab 3), do only the children less than 5 have them? </font>

## Forgotten Roster Variables <a id='forgotten'></a>

In Lab 1, we may not have not included in all the variables we needed in the csv file. Check to make sure that your roster has `Relationship to Household Head`, `Mother ID`, and `Father ID`. If not, join `combined` with your original data set to get those columns.

In [ ]:
# Read in original csv
original = ...
# Select columns
original = ...
# Rename variables
original = ...
### STUDENT SOLUTION

Generate a Master ID to join on.

In [ ]:
original['Master ID'] = ...
### STUDENT SOLUTION

Perform a left join with `combined` and the original data set, copying the procedure from above.

In [ ]:
combined = ...
### STUDENT SOLUTION

## Joining External Data <a id='outside'></a>

Now for the <i>exciting</i> part!  Joining in data that we have not already prepared for you!!!

Go to your country's survey on the LSMS site and select "Data" instead of "Documentation".  
Sign the data use agreement.

<font color="blue"> Item 3: Write what you have promised to do in the agreement. </font>

You will likely have to download all data (in a zip file).  Using your questionaire or codebook, determine which of these data sets include the education & income variables.  Note that there may be two files needed for the education variables - one for children & one for adults.  

***Talk to me to confirm that you have selected the right variables & files.***

<font color="blue"> Item 4: What are the names of the files you are uploading?

Upload the Stata (.dta) data sets to Jupyter (into the same folder that contains this lab). 

The data sets are in Stata format (.dta). Stata is a statistical software commonly used by economists and researchers in public health. We use the `read_stata` function from `pandas` to read read the data into a dataframe, that you can then turn into a `datascience` table. 

Plug in the name of a data set you want to read into the code below. You'll need to repeat the process for any additional data sets you want to read in.

In [ ]:
# replace the file name with that of the one in your server
# fillna(-99) replaces missing values with -99, which are easier for us to work with than Python's "nan"

df = pd.read_stata('education.dta', convert_categoricals=False)
filled_nans = df.fillna(-99)
additional_table = Table.from_df(filled_nans)

additional_table.show(20)

For each table, select the variables Household ID, Individual ID, and the variables you identified (income/schooling). 

In [ ]:
### STUDENT SOLUTION

Now rename the columns accordingly. Make sure to check the values in your table to make sure they are what you expected.

In [ ]:
### STUDENT SOLUTION

Make a `Master ID` column so you can join this table to your `combined` table.

In [ ]:
### STUDENT SOLUTION

Now use a **left join** to add `additional_table` onto `combined`.

In [ ]:
### STUDENT SOLUTION

### <font color="blue"> Item 5-??: Sanity Checks
Each time you add a data set to the `combined` table, confirm the following:<br><br>

<font color="blue"> a) Are there still the same number of people in your data as on the original roster (Lab 1)?<br><br>


 
b) If you examine the new variables just joined in, are the people that you expect to have values for these variables whom you expect? (For example, are only children enrolled in school?  Are mainly adults receiving salaries?)</font>

Save your file as `Lab_4.csv`.

In [ ]:
### STUDENT SOLUTION
combined.to_csv('Lab_4.csv')

### <font color="blue"> Last Item
Email your Lab_4.csv to sar.48@berkeley.edu  Let me know what country it is in the email.  I will take a look to make sure your data looks ok! 

Extra time?  Join/add in the variables you will use in your final paper.